In [120]:
# Carga de librerías
import pandas as pd

Lectura de los ficheros de lso cuales se van a obtener los datos:
* **cv19_municipio-asignacion_casos.csv:** fichero con los casos de covid según el centro de atención primaria en el que se diagnosticó el positivo.
* **cv19_municipio-residencia_casos.csv:** fichero con los casos de covid según la residencia registrada en tarjeta sanitaria del paciente.
* **municipios_20170101.csv:** fichero con los datos geográficos de los municipios de canarias

In [179]:
#Lectura de los datos
data_asignacion = pd.read_csv("data_ini/cv19_municipio-asignacion_casos.csv")
data_residencia = pd.read_csv("data_ini/cv19_municipio-residencia_casos.csv")
data_coordenadas_municipios = pd.read_csv("data_ini/municipios_20170101.csv")
data_coordenadas_islas = pd.read_csv("data_ini/islas_20170101.csv")

Visualización de los datos de covid leidos de los ficheros de datos.

In [122]:
data_asignacion.head()

,fecha_datos,isla,municipio,sexo,grupo_edad,fecha_caso,fecha_fallecido,fecha_curado,estado_caso
0,14/03/2022,TENERIFE,Adeje,Hombre,30 - 39,02/01/2022,NaN,13/01/2022,Cerrado por alta médica
1,14/03/2022,TENERIFE,Adeje,Hombre,30 - 39,08/01/2022,NaN,19/01/2022,Cerrado por alta médica
2,14/03/2022,TENERIFE,Adeje,Hombre,40 - 49,06/01/2022,NaN,17/01/2022,Cerrado por alta médica
3,14/03/2022,TENERIFE,Adeje,Hombre,50 - 59,02/01/2022,NaN,13/01/2022,Cerrado por alta médica
4,14/03/2022,TENERIFE,Adeje,Hombre,40 - 49,08/01/2022,NaN,19/01/2022,Cerrado por alta médica


In [123]:
data_residencia.head()

,fecha_datos,isla,municipio,sexo,grupo_edad,fecha_caso,fecha_fallecido,fecha_curado,estado_caso
0,14/03/2022,TENERIFE,Adeje,Mujer,20 - 29,02/01/2022,NaN,13/01/2022,Cerrado por alta médica
1,14/03/2022,TENERIFE,SIN ESPECIFICAR,Mujer,20 - 29,02/01/2022,NaN,13/01/2022,Cerrado por alta médica
2,14/03/2022,TENERIFE,Adeje,Mujer,10 - 19,08/01/2022,NaN,19/01/2022,Cerrado por alta médica
3,14/03/2022,TENERIFE,SIN ESPECIFICAR,Hombre,30 - 39,02/01/2022,NaN,13/01/2022,Cerrado por alta médica
4,14/03/2022,TENERIFE,Adeje,Hombre,30 - 39,08/01/2022,NaN,19/01/2022,Cerrado por alta médica


Identificamos cada uno de los conjuntos de datos con Asignación y Residencia para en pasos posteriores fusionar ámbos conjuntos de datos en uno y tener identificados cada uno de los casos.

In [124]:
data_asignacion['identificacion_caso'] = 'Asignación'
data_residencia['identificacion_caso'] = 'Residencia'

In [125]:
# Fusión de los conjuntos de datos en uno
data = pd.concat([data_asignacion, data_residencia])

Comprobamos la existencia de valores nulos en el conjunto de datos.

In [126]:
data.isnull().sum()

fecha_datos                 0
isla                        0
municipio                   0
sexo                        0
grupo_edad                  0
fecha_caso                  0
fecha_fallecido        563766
fecha_curado            28688
estado_caso                 0
identificacion_caso         0
dtype: int64

Podemos observar que nos encontramos con valores nulos en los atributos fecha_fallecido y fecha_curado. En este caso lo consideramos como valores válidos ya que la fecha_fallecido nula hace referencia a aquellas personas que no han fallecido, ya sea porque se han curado o porque aún están enfermos. Por otro lado, los que encontramos con fecha_curado a nulo hace referencia a aquellas personas que todavía tienen la enfermedad y no se han curado o personas fallecidas.

Generamos 3 nuevas variables que nos van a indicar si el paciente está fallecido, curado o continúa enfermo en función de la fecha de fallecimiento y la fecha de curación.

In [142]:
#Nuevas variables
data['fallecido'] = None
data['curado'] = None
data['continua_enfermo'] = None

In [143]:
data.loc[data['fecha_fallecido'].isna(), 'fallecido'] = 'No'
data.loc[data['fecha_fallecido'].notna(), 'fallecido'] = 'Sí'

In [144]:
data.loc[data['fecha_curado'].isna(), 'curado'] = 'No'
data.loc[data['fecha_curado'].notna(), 'curado'] = 'Sí'

In [145]:
data.loc[data['fecha_curado'].isna() & data['fecha_fallecido'].isna(), 'continua_enfermo'] = 'Sí'
data.loc[data['fecha_curado'].notna() | data['fecha_fallecido'].notna(), 'continua_enfermo'] = 'No'

In [146]:
data.head()

,fecha_datos,isla,municipio,sexo,grupo_edad,fecha_caso,fecha_fallecido,fecha_curado,estado_caso,identificacion_caso,fallecido,curado,continua_enfermo
0,14/03/2022,TENERIFE,Adeje,Hombre,30 - 39,02/01/2022,NaN,13/01/2022,Cerrado por alta médica,Asignación,No,Sí,No
1,14/03/2022,TENERIFE,Adeje,Hombre,30 - 39,08/01/2022,NaN,19/01/2022,Cerrado por alta médica,Asignación,No,Sí,No
2,14/03/2022,TENERIFE,Adeje,Hombre,40 - 49,06/01/2022,NaN,17/01/2022,Cerrado por alta médica,Asignación,No,Sí,No
3,14/03/2022,TENERIFE,Adeje,Hombre,50 - 59,02/01/2022,NaN,13/01/2022,Cerrado por alta médica,Asignación,No,Sí,No
4,14/03/2022,TENERIFE,Adeje,Hombre,40 - 49,08/01/2022,NaN,19/01/2022,Cerrado por alta médica,Asignación,No,Sí,No


In [147]:
data.isnull().sum()

fecha_datos                 0
isla                        0
municipio                   0
sexo                        0
grupo_edad                  0
fecha_caso                  0
fecha_fallecido        563766
fecha_curado            28688
estado_caso                 0
identificacion_caso         0
fallecido                   0
curado                      0
continua_enfermo            0
dtype: int64

Generamos una nueva variable para distinguir la provincia a la que pertenece cada uno de los casos de covid registrados.

In [148]:
islas_sc = ['TENERIFE', 'LA GOMERA', 'LA PALMA', 'EL HIERRO']
islas_lp = ['GRAN CANARIA', 'FUERTEVENTURA', 'LANZAROTE']

data['provincia'] = None

for i in range(len(islas_sc)):
    data.loc[data.isla == islas_sc[i], 'provincia'] = 'Santa Cruz de Tenerife'
    
for j in range(len(islas_lp)):
    data.loc[data.isla == islas_lp[j], 'provincia'] = 'Las Palmas'

Generamos nuevas variables para determinar la localización aproximada de donde se localiza el caso de covid, en este caso emplearemos las coordenadas del municipio, por lo que se obtienen del fichero de municipios de canarias los datos de latitud y longitud de cada municipio de cada caso. A su vez también se obtiene la superficie de cada uno de los municipios en kilómetros cuadrados.

In [149]:
data['latitud_municipio'] = None
data['longitud_municipio'] = None
data['superficie_municipio'] = None

In [152]:
for i in range(len(data_coordenadas_municipios)):
    data.loc[data.municipio == data_coordenadas_municipios.iloc[i].etiqueta, 'latitud_municipio'] = data_coordenadas_municipios.iloc[i].latitud
    data.loc[data.municipio == data_coordenadas_municipios.iloc[i].etiqueta, 'longitud_municipio'] = data_coordenadas_municipios.iloc[i].longitud
    data.loc[data.municipio == data_coordenadas_municipios.iloc[i].etiqueta, 'superficie_municipio'] = data_coordenadas_municipios.iloc[i].ign_sup

In [153]:
data.head()

,fecha_datos,isla,municipio,sexo,grupo_edad,fecha_caso,fecha_fallecido,fecha_curado,estado_caso,identificacion_caso,fallecido,curado,continua_enfermo,provincia,latitud_municipio,longitud_municipio,superficie_municipio
0,14/03/2022,TENERIFE,Adeje,Hombre,30 - 39,02/01/2022,NaN,13/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife,28.139453,-16.739477,10595.2
1,14/03/2022,TENERIFE,Adeje,Hombre,30 - 39,08/01/2022,NaN,19/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife,28.139453,-16.739477,10595.2
2,14/03/2022,TENERIFE,Adeje,Hombre,40 - 49,06/01/2022,NaN,17/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife,28.139453,-16.739477,10595.2
3,14/03/2022,TENERIFE,Adeje,Hombre,50 - 59,02/01/2022,NaN,13/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife,28.139453,-16.739477,10595.2
4,14/03/2022,TENERIFE,Adeje,Hombre,40 - 49,08/01/2022,NaN,19/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife,28.139453,-16.739477,10595.2


## Fichero de municipios

In [155]:
data_coordenadas_municipios.head()

,geocode,geopadre,etiqueta,notas,granularidad,gcd_provincia,gcd_isla,gcd_grancomarca,gcd_comarca,ign_sup,ign_perim,utm_x,utm_y,longitud,latitud,utm_x_capi,utm_y_capi,long_capi,lati_capi
0,35001,ES705A22,Agaete,Ayuntamiento de Agaete,MUNICIPIOS,ES701,ES705,ES705A2,ES705A22,4451.16,41038,432237.32,3105495.86,-15.689645,28.073135,431360.28,3108387.71,-15.698739,28.099193
1,35002,ES705A32,Agüimes,Ayuntamiento de Agüimes,MUNICIPIOS,ES701,ES705,ES705A3,ES705A32,7889.72,55508,455417.29,3085973.99,-15.453004,27.897893,456117.58,3086639.88,-15.445914,27.903927
2,35003,ES704A01,Antigua,Ayuntamiento De Antigua,MUNICIPIOS,ES701,ES704,ES704A0,ES704A01,25024.80,79292,603336.94,3136732.92,-13.945579,28.352768,596564.74,3144502.79,-14.014024,28.423412
3,35004,ES708A01,Arrecife,Ayuntamiento De Arrecife,MUNICIPIOS,ES701,ES708,ES708A0,ES708A01,2427.54,36641,641157.78,3206510.34,-13.551117,28.978878,641613.10,3204338.91,-13.546719,28.959235
4,35005,ES705A23,Artenara,Ayuntamiento de Artenara,MUNICIPIOS,ES701,ES705,ES705A2,ES705A23,6641.90,50791,432859.41,3099392.63,-15.682966,28.018075,436451.97,3099620.39,-15.646436,28.020308


In [173]:
data_municipios = pd.DataFrame()

data_municipios = data_coordenadas_municipios.loc[:,['geocode', 'etiqueta', 'ign_sup', 'ign_perim', 'longitud', 'latitud', 'long_capi', 'lati_capi']]

In [174]:
data_municipios.head()

,geocode,etiqueta,ign_sup,ign_perim,longitud,latitud,long_capi,lati_capi
0,35001,Agaete,4451.16,41038,-15.689645,28.073135,-15.698739,28.099193
1,35002,Agüimes,7889.72,55508,-15.453004,27.897893,-15.445914,27.903927
2,35003,Antigua,25024.80,79292,-13.945579,28.352768,-14.014024,28.423412
3,35004,Arrecife,2427.54,36641,-13.551117,28.978878,-13.546719,28.959235
4,35005,Artenara,6641.90,50791,-15.682966,28.018075,-15.646436,28.020308


In [175]:
data_municipios = data_municipios.rename(columns = {'etiqueta':'municipio', 'ign_sup':'sueprficie', 'ign_perim':'perimetro', 'long_capi':'longitud_capital', 'lati_capi':'latitud_capital'})

,geocode,municipio,sueprficie,perimetro,longitud,latitud,longitud_capital,latitud_capital
0,35001,Agaete,4451.16,41038,-15.689645,28.073135,-15.698739,28.099193
1,35002,Agüimes,7889.72,55508,-15.453004,27.897893,-15.445914,27.903927
2,35003,Antigua,25024.80,79292,-13.945579,28.352768,-14.014024,28.423412
3,35004,Arrecife,2427.54,36641,-13.551117,28.978878,-13.546719,28.959235
4,35005,Artenara,6641.90,50791,-15.682966,28.018075,-15.646436,28.020308
...,...,...,...,...,...,...,...,...
83,38050,Vallehermoso,10854.20,81938,-17.266338,28.133298,-17.266460,28.179321
84,38051,La Victoria de Acentejo,1808.63,21800,-16.438806,28.416691,NaN,NaN
85,38052,Vilaflor de Chasna,5627.11,34914,-16.647254,28.159079,-16.636767,28.158929
86,38053,Villa de Mazo,7067.20,46447,-17.798915,28.576007,-17.779602,28.606118


## Ficchero islas

In [180]:
data_coordenadas_islas.head()

,geocode,etiqueta,notas,granularidad,gcd_provincia,superficie,perimetro,utm_x,utm_y,longitud,latitud,utm_x_capi,utm_y_capi,long_capi,lati_capi
0,ES703,El Hierro,Cabildo de El Hierro,ISLAS,ES702,26771.3,120433,200495.81,3072499.65,-18.038165,27.743606,212755,3079276,-17.915572,27.807376
1,ES704,Fuerteventura,Cabildo de Fuerteventura,ISLAS,ES701,166100.2,345150,597480.87,3141956.64,-14.004885,28.400363,611517,3152969,-13.860557,28.498627
2,ES705,Gran Canaria,Cabildo de Gran Canaria,ISLAS,ES701,156082.3,281460,440369.39,3092715.10,-15.606239,27.958157,458747,3109221,-15.419985,28.107852
3,ES706,La Gomera,Cabildo de La Gomera,ISLAS,ES702,36760.4,110636,281385.63,3112411.99,-17.225583,28.119230,292459,3109118,-17.112333,28.091298
4,ES707,La Palma,Cabildo de La Palma,ISLAS,ES702,70610.2,181891,220710.17,3173129.57,-17.857377,28.655223,229746,3175822,-17.765661,28.681415


In [182]:
data_islas = pd.DataFrame()

data_islas = data_coordenadas_islas.loc[:,['geocode', 'etiqueta', 'superficie', 'perimetro', 'longitud', 'latitud', 'long_capi', 'lati_capi']]

In [189]:
data_islas = data_islas.rename(columns = {'etiqueta':'isla','long_capi':'longitud_capital', 'lati_capi':'latitud_capital'})

In [196]:
data_islas['isla'] = data_islas['isla'].str.upper()

data_islas['provincia'] = None

for i in range(len(islas_sc)):
    data_islas.loc[data_islas.isla == islas_sc[i], 'provincia'] = 'Santa Cruz de Tenerife'
    
for j in range(len(islas_lp)):
    data_islas.loc[data_islas.isla == islas_lp[j], 'provincia'] = 'Las Palmas'

print(data_islas)

  geocode           isla  superficie  perimetro   longitud    latitud  \
0   ES703      EL HIERRO     26771.3     120433 -18.038165  27.743606   
1   ES704  FUERTEVENTURA    166100.2     345150 -14.004885  28.400363   
2   ES705   GRAN CANARIA    156082.3     281460 -15.606239  27.958157   
3   ES706      LA GOMERA     36760.4     110636 -17.225583  28.119230   
4   ES707       LA PALMA     70610.2     181891 -17.857377  28.655223   
5   ES708      LANZAROTE     84488.5     288011 -13.636218  29.039468   
6   ES709       TENERIFE    203446.3     384234 -16.621481  28.293378   

   longitud_capital  latitud_capital               provincia  
0        -17.915572        27.807376  Santa Cruz de Tenerife  
1        -13.860557        28.498627              Las Palmas  
2        -15.419985        28.107852              Las Palmas  
3        -17.112333        28.091298  Santa Cruz de Tenerife  
4        -17.765661        28.681415  Santa Cruz de Tenerife  
5        -13.563186        28.958014 

## Escritura de ficheros

In [ ]:
data.to_csv('data_fin/covid_canarias.csv', index=False)
data_municipios.to_csv('data_fin/municipios.csv', index=False)
data_islas.to_csv('data_fin/islas.csv', index=False)